## Import Data

In [3]:
# Import helper functions
# for file/directory operations
import os
import random
# for file copying/moving
import shutil

# Function to split directory into train, test and validation sets
# Default split is 70% train, 20% test, 10% validation
# Directory structure should be: directory/class_name/file_name.jpg
def split_dir_to_train_test_val(directory = "tsrd-train/",
                            train_size = 0.7,
                            test_size = 0.2,
                            val_size = 0.1):

  # Set random seed
  rng = random.Random(42)

  for root, folders, files in os.walk(directory):
    for folder in folders:
      # Create list of the files
      list_of_files = []
      for file_name in os.listdir(root+folder+"/"):
        list_of_files.append(file_name)

      #  Shuffle the list
      rng.shuffle(list_of_files)

      # Create lists of files
      train_files = list_of_files[:int(len(list_of_files)*train_size)]
      # we put first 70% files as train_files
      test_files = list_of_files[int(len(list_of_files)*train_size) : int(len(list_of_files)*(train_size+test_size))]
      # we put 20% files as test_files
      val_files = list_of_files[int(len(list_of_files)*(train_size+test_size)):]

      # Create folders and files for train data
      for one_file in train_files:

        # Copy  files
        dest_dir = "files/train/"+folder+"/"
        os.makedirs(dest_dir, exist_ok=True)

        shutil.copy2(src=(root+folder+"/"+one_file),
                    dst=(dest_dir+one_file))
      print(f"Folder {folder}. Train data copied. {len(train_files)} files")

      # Create folders and files for test data
      for one_file in test_files:
        # Copy  files
        dest_dir = "files/test/"+folder+"/"
        os.makedirs(dest_dir, exist_ok=True)

        shutil.copy2(src=(root+folder+"/"+one_file),
                    dst=(dest_dir+one_file))
      print(f"Folder {folder}. Test data copied. {len(test_files)} files")

      # Create folders and files for validation data
      for one_file in val_files:

        # Copy  files
        dest_dir = "files/validation/"+folder+"/"
        os.makedirs(dest_dir, exist_ok=True)

        shutil.copy2(src=(root+folder+"/"+one_file),
                    dst=(dest_dir+one_file))
      print(f"Folder {folder}. Validation data copied. {len(val_files)} files")




def get_class_names_from_folder(directory):

  import pathlib
  import numpy as np
  data_dir = pathlib.Path(directory)
  class_names = np.array(sorted([item.name for item in data_dir.glob("*")])) # Created a list of class names
  return class_names
  print(class_names)

In [5]:
# Split images dir to train, test and validation
split_dir_to_train_test_val(directory="finaldataset/",
                            train_size=0.7,
                            test_size=0.2,
                            val_size=0.1)

## Explore and visualize the data

In [8]:
# Using helper function get class names
class_names = get_class_names_from_folder(directory="files/train/")
class_names

array([], dtype=float64)

In [9]:
len(class_names)

0

In [11]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
import random

def visualize_random_image(target_dir, target_class):
  image_dir = target_dir + target_class
  random_image = random.sample(os.listdir(image_dir), 1)

  img = mpimg.imread(image_dir + "/" + random_image[0])
  plt.imshow(img)
  plt.title(target_class)

visualize_random_image("files/test/", "007")

FileNotFoundError: [Errno 2] No such file or directory: 'files/test/007'

## Create datasets from directories

In [10]:
# Import library
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)
val_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(directory="files/train",
                                               target_size=(240, 240),
                                               batch_size=32,
                                               class_mode="categorical")

test_data = test_datagen.flow_from_directory(directory="files/test",
                                               target_size=(240, 240),
                                               batch_size=32,
                                               class_mode="categorical")

validation_data = val_datagen.flow_from_directory(directory="files/validation",
                                               target_size=(240, 240),
                                               batch_size=32,
                                               class_mode="categorical")

Found 1987 images belonging to 10 classes.
Found 567 images belonging to 10 classes.
Found 290 images belonging to 10 classes.


## Create and fit the model

In [1]:
# Import library
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)
val_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(directory="files/train",
                                               target_size=(240, 240),
                                               batch_size=32,
                                               class_mode="categorical")

test_data = test_datagen.flow_from_directory(directory="files/test",
                                               target_size=(240, 240),
                                               batch_size=32,
                                               class_mode="categorical")

validation_data = val_datagen.flow_from_directory(directory="files/validation",
                                               target_size=(240, 240),
                                               batch_size=32,
                                               class_mode="categorical")

"""## Create and fit the model"""


# Set random seed
tf.random.set_seed(42)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

# Create model
model = Sequential([
    Conv2D(16, 3, activation="relu", input_shape=(240, 240, 3)),
    MaxPool2D(pool_size=2),
    Conv2D(16, 3, activation="relu"),
    MaxPool2D(pool_size=2),
    Flatten(),
    Dense(10, activation="softmax")
])




# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(),
              metrics=["accuracy"])

model.summary()




# Fit the model

'''
batch_sizes = 32
epoch = 3
trainsteps = (train_data.n//batch_sizes)
valsteps = (validation_data.n//batch_sizes)
print(train_data.n)
print(validation_data.n)
print(test_data.n)
print(trainsteps)
print(valsteps)
print(train_data)

history = model.fit(train_data,
                    batch_size=batch_sizes,
                    epochs=epoch,
                    steps_per_epoch=trainsteps,
                    validation_data=validation_data,
                    validation_steps=valsteps)




# Evaluate model


model.evaluate(test_data)

import pandas as pd

pd.DataFrame(history.history).plot()

model.save('my_model.keras')
'''


2024-08-15 11:12:48.312962: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


FileNotFoundError: [Errno 2] No such file or directory: 'files/train'

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

new_model = tf.keras.models.load_model('my_model.keras')

image_path = 'files/test/055/055_1_0007.png'  # Replace with the actual image path
image = load_img(image_path, target_size=(240, 240))
image_array = img_to_array(image)
image_array = image_array / 255.0  # Normalize pixel values to [0, 1]
image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension


# Make predictions using the loaded model
predictions = new_model.predict(image_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Print the predicted class index and corresponding class label
print('Predicted Class Index:', predicted_class_index)

# Assuming you have a list of class labels
class_labels = ['007','011','012','028','030','035','050','054','055','056']  # Replace with your class labels
predicted_class_label = class_labels[predicted_class_index]

print('Predicted Class Label:', predicted_class_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Predicted Class Index: 8
Predicted Class Label: 055


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

new_model = tf.keras.models.load_model('my_model.keras')

image_path = 'files/test/055/055_1_0007.png'  # Replace with the actual image path
image = load_img(image_path, target_size=(240, 240))
image_array = img_to_array(image)
image_array = image_array / 255.0  # Normalize pixel values to [0, 1]
image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

# Make predictions using the loaded model
predictions = new_model.predict(image_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Get the confidence for the predicted class
confidence = predictions[0][predicted_class_index] * 100  # Convert to percentage

# Print the predicted class index, corresponding class label, and prediction confidence
print('Predicted Class Index:', predicted_class_index)
class_labels = ['007', '011', '012', '028', '030', '035', '050', '054', '055', '056']  # Replace with your class labels
predicted_class_label = class_labels[predicted_class_index]
print('Predicted Class Label:', predicted_class_label)
print('Prediction Confidence (%):', confidence)

# Get softmax values for all categories
softmax_output = tf.nn.softmax(predictions)

# Convert softmax output to a numpy array for easier manipulation
softmax_values = softmax_output.numpy()

# Print softmax values for all categories
print(softmax_values)
print(softmax_values[0][predicted_class_index])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted Class Index: 8
Predicted Class Label: 055
Prediction Confidence (%): 99.99905824661255
[[0.08533686 0.08533686 0.08533687 0.08533686 0.08533686 0.08533686
  0.08533686 0.08533686 0.23196746 0.08533766]]
0.23196746


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

new_model = tf.keras.models.load_model('my_model.keras')
new_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 238, 238, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 119, 119, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 117, 117, 16)   │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 58, 58, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │       538,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,623,056 (6.19 MB)

 Trainable params: 541,018 (2.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,082,038 (4.13 MB)